In [1]:
import os
import cv2
import glob
import shutil
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

def getImagesInDir(dir_path):
    img_formats = ['bmp', 'jpg', 'jpeg', 'png', 'tif', 'tiff', 'dng']  # acceptable image suffixes
    image_list = []
    for img_format in img_formats:
        for filename in glob.glob(dir_path + f'/*.{img_format}'):
            image_list.append(filename)
    return image_list

In [2]:
classes = ['pedestrian']
img_dir = 'JPEGImages/'
ann_dir = 'Annotations/'
image_paths = getImagesInDir(img_dir)
image_paths = sorted(image_paths)

In [3]:
submission = pd.read_csv('sample_submission.csv')
for img_path in tqdm(image_paths, total=len(image_paths)):
    # read image
    img = cv2.imread(img_path)
    
    basename = os.path.basename(img_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(ann_dir + '/' + basename_no_ext + '.xml')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        reid = obj.find('reid').text[:-2]
        submission.loc[len(submission)] = [basename_no_ext,
                                           reid,
                                           '1',
                                           int(xmlbox.find('xmin').text)/w,
                                           int(xmlbox.find('ymin').text)/h,
                                           int(xmlbox.find('xmax').text)/w,
                                           int(xmlbox.find('ymax').text)/h]

100%|██████████| 153/153 [00:16<00:00,  9.31it/s]


In [4]:
submission.to_csv('answer.csv', index=False)

In [5]:
import numpy as np
len(np.unique(submission['reid']))

67